<a href="https://colab.research.google.com/github/Akasxh/Stock_Analysis/blob/main/edgar_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install beautifulsoup4

!pip install sec-edgar-downloader

!pip install openpyxl

!pip install arelle

!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00


In [8]:
from sec_edgar_downloader import Downloader

import os
import glob
import re
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt


In [14]:
# Initialize the downloader
dl = Downloader("YOUR_COMPANY_NAME", "YOUR_EMAIL_ADDRESS","/content")

In [6]:
def extract_text_from_html(file_path):
    # Ensure the file exists
    if not os.path.exists(file_path):
        return "File does not exist"

    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract text from the parsed HTML
    text = soup.get_text(separator='\n', strip=True)

    return text

In [23]:
def truncate_and_overwrite(file_path, target_line='XBRL INSTANCE DOCUMENT'):
    """
    Truncates the file content after a specific target line and overwrites the original file.

    :param file_path: Path to the file to be modified.
    :param target_line: The line after which all text should be discarded.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as infile:
            lines = infile.readlines()

        # Determine the point to truncate
        truncation_point = None
        for i, line in enumerate(lines):
            if target_line in line:
                truncation_point = i + 1  # Include the target line and stop after
                break

        # If the target line is found, truncate the lines
        if truncation_point is not None:
            lines = lines[:truncation_point]
            # Overwrite the original file with truncated content
            with open(file_path, 'w', encoding='utf-8') as outfile:
                outfile.writelines(lines)
        else:
            print(f"Target line '{target_line}' not found in file.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [26]:
def merge(company_ticker):
    dl.get("10-K", company_ticker.upper(), after="1995-01-01", before="2023-01-01", download_details=False)
    base_directory = f'/content/sec-edgar-filings/{company_ticker}/10-K'
    file_paths = glob.glob(os.path.join(base_directory, '**', '*.txt'), recursive=True)

    for file_path in file_paths:
            truncate_and_overwrite(file_path)

    #output_filename = f'/content/merged_output_{company_ticker.upper()}.txt'
    '''with open(output_filename, 'w') as output_file:
        for file_path in file_paths:
            truncate_and_overwrite(file_path)
            #extracted_data = extract_text_from_html(file_path)
            #output_file.write(extracted_data + '\n')'''


In [17]:
def visualise(company_ticker):
    merge(company_ticker)

In [18]:
def main():
    company_ticker = input("Enter the company ticker: ")
    visualise(company_ticker)

In [19]:
main()

Enter the company ticker: NFLX


In [24]:
truncate_and_overwrite("/content/sec-edgar-filings/NFLX/10-K/0001065280-15-000006/full-submission.txt")

In [9]:
# Title for the application
st.title("Financial Analysis Dashboard")

# Sidebar for API key input
st.sidebar.markdown("## Setup")
api_key_input = st.sidebar.text_input(
    "OpenAI API Key",
    type="password",
    placeholder="Paste your OpenAI API key here (sk-...)",
    help="You can get your API key from https://platform.openai.com/account/api-keys."
)

# Sidebar for entering the company ticker
company_ticker = st.sidebar.text_input(
    "Enter Company Ticker",
    placeholder="E.g., AAPL, GOOGL",
    help="Enter the ticker symbol of the company you wish to analyze."
)

# Placeholder for displaying graphs and analysis
st.sidebar.markdown("---")
data_display = st.empty()
graph_display = st.empty()

# Main function that uses the API to perform analysis and fetch data
def perform_analysis(api_key, ticker):
    if not api_key or not ticker:
        st.error("Please enter both your OpenAI API key and the company ticker.")
        return

    # Example of fetching and processing data (mock-up)
    # In reality, you would use the OpenAI API here to fetch and process the company's data
    data = pd.DataFrame({
        'Date': pd.date_range(start='1/1/2020', periods=100),
        'Value': pd.np.random.rand(100).cumsum()
    })

    # Display the data
    data_display.dataframe(data)

    # Plotting a graph of the data
    plt.figure(figsize=(10, 4))
    plt.plot(data['Date'], data['Value'], marker='o', linestyle='-')
    plt.title('Company Financial Performance Over Time')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Show the graph
    graph_display.pyplot(plt)

# Button to trigger the analysis
if st.sidebar.button('Analyze'):
    perform_analysis(api_key_input, company_ticker)


2024-05-08 14:56:11.195 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-05-08 14:56:11.199 Session state does not function when running a script without `streamlit run`
